<a href="https://colab.research.google.com/github/22f3002718/ai-notebooks/blob/main/starcoder2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers accelerate


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "bigcode/starcoder2-3b"

# Load tokenizer (converts text to tokens and back)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load model (uses float16 to run efficiently on Colab GPUs)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",           # Automatically puts it on GPU
    torch_dtype=torch.float16    # Efficient precision
)
model.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/12.1G [00:00<?, ?B/s]

Starcoder2ForCausalLM(
  (model): Starcoder2Model(
    (embed_tokens): Embedding(49152, 3072)
    (layers): ModuleList(
      (0-29): 30 x Starcoder2DecoderLayer(
        (self_attn): Starcoder2Attention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=True)
          (k_proj): Linear(in_features=3072, out_features=256, bias=True)
          (v_proj): Linear(in_features=3072, out_features=256, bias=True)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=True)
        )
        (mlp): Starcoder2MLP(
          (c_fc): Linear(in_features=3072, out_features=12288, bias=True)
          (c_proj): Linear(in_features=12288, out_features=3072, bias=True)
          (act): PytorchGELUTanh()
        )
        (input_layernorm): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
      )
    )
    (norm): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
    (rot

In [ ]:
prompt = "def is_prime(n):\n    \"\"\"Check if a number is prime.\"\"\"\n"

# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate complete function
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,                     # enough tokens for logic
        do_sample=True,                         # enables randomness
        temperature=0.7,                        # balance creativity
        top_k=50,
        eos_token_id=tokenizer.eos_token_id,    # stop properly
        pad_token_id=tokenizer.eos_token_id     # avoid warning
    )

# Decode the output tokens to readable text
generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print only the generated part, ignoring repeat of the prompt
print("\n🧠 Generated Code:\n")
print(generated_code[len(prompt):].strip())



🧠 Generated Code:

#     if n==2:
#         return True
#     elif n%2==0:
#         return False
#     else:
#         for i in range(3,int(n**0.5)+1,2):
#             if n%i==0:
#                 return False
#         return True
    return all(n%i for i in range(3,int(n**0.5)+1,2)) and n%2!=0


# +
def is_prime(n):
    """Check if a number is prime."""


In [ ]:
def run(prompt, max_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_tokens, do_sample=False)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.strip()

# 1️⃣ LENGTH & LATENCY
print("🔍 Length + Latency Test")
for length in [128, 256, 512, 768, 1024]:
    test_prompt = "def example():\n    # " + "This is a comment.\n    # " * (length // 10)
    try:
        start = time.time()
        output = run(test_prompt, max_tokens=20)
        end = time.time()
        print(f"✅ {length} tokens → Success | ⏱ {end - start:.2f}s")
    except Exception as e:
        print(f"❌ {length} tokens → Failure | 💥 {e}")
        break

# 2️⃣ COMPLEXITY TEST
print("\n🔍 Complexity Prompt Test")
prompts = [
    "# Add two numbers",                           # simple
    "# Do something useful",                       # vague
    "# Implement greatness",                       # abstract
]

for p in prompts:
    out = run(p, max_tokens=60)
    success = "def" in out or "=" in out
    status = "✅ Success" if success else "❌ Failure"
    print(f"\nPrompt: {p}\n{status}\n📝 Output:\n{out}")

# 3️⃣ STABILITY TEST (minor changes to prompt)
print("\n🔍 Prompt Stability Test")
base = "def reverse_string(s):"
variation = "def reverse_str(s):"

out1 = run(base, 60)
out2 = run(variation, 60)

same = out1.strip().splitlines()[:2] == out2.strip().splitlines()[:2]
status = "✅ Stable" if same else "❌ Unstable"
print(f"\nPrompt 1: {base}\n📝 {out1}\n\nPrompt 2: {variation}\n📝 {out2}\n{status}")

# 4️⃣ AUTOCOMPLETION TEST
print("\n🔍 Code Autocompletion Test")
prompt = "def factorial(n):\n    \"\"\"Returns factorial\"\"\"\n    if n == 0:"
out = run(prompt, 60)
success = "return" in out and "*" in out
status = "✅ Success" if success else "❌ Failure"
print(f"Prompt:\n{prompt}\n\n📝 Output:\n{out}\n{status}")

# 5️⃣ SYNTAX CHECK TEST (does it produce valid Python code?)
print("\n🔍 Syntax Check Test")

import ast

def is_valid_syntax(code):
    try:
        ast.parse(code)
        return True
    except:
        return False

prompt = "def is_even(n):\n    return n % 2 == 0"
output = run(prompt, 10)
syntax_ok = is_valid_syntax(output)
status = "✅ Valid Python" if syntax_ok else "❌ Invalid"
print(f"\n📝 Output:\n{output}\n{status}")


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


🔍 Length + Latency Test
❌ 128 tokens → Failure | 💥 name 'time' is not defined

🔍 Complexity Prompt Test


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Prompt: # Add two numbers
✅ Success
📝 Output:
# Add two numbers
def add(x, y):
    return x + y

# Subtract two numbers
def subtract(x, y):
    return x - y

# Multiply two numbers
def multiply(x, y):
    return x * y

# Divide two numbers
def


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...


Prompt: # Implement greatness
✅ Success
📝 Output:
# Implement greatness

# +
# Implement greatness

def greatness(x, y):
    if x > y:
        return 1
    elif x == y:
        return 0
    else:
        return -1


# -

# Test greatness
greatness(

🔍 Prompt Stability Test


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Prompt 1: def reverse_string(s):
📝 def reverse_string(s):
    return s[::-1]

def is_palindrome(s):
    return s == reverse_string(s)

def is_palindrome_recursive(s):
    if len(s) <= 1:
        return True
    else:
        return s[0

Prompt 2: def reverse_str(s):
📝 def reverse_str(s):
    return s[::-1]

def is_palindrome(s):
    return s == reverse_str(s)

def is_palindrome_v2(s):
    return s == s[::-1]

def is_palindrome_v3(
❌ Unstable

🔍 Code Autocompletion Test


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt:
def factorial(n):
    """Returns factorial"""
    if n == 0:

📝 Output:
def factorial(n):
    """Returns factorial"""
    if n == 0:
        return 1
    else:
        return n * factorial(n - 1)


def fibonacci(n):
    """Returns fibonacci"""
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return
✅ Success

🔍 Syntax Check Test

📝 Output:
def is_even(n):
    return n % 2 == 0

def is_odd(n):
❌ Invalid


In [1]:
# ✅ Install Transformers if not already
!pip install -q transformers accelerate

# ✅ Load StarCoder2-3B from Hugging Face
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "bigcode/starcoder2-3b"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# ✅ Code generation function
def generate_code(prompt, max_new_tokens=128):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# ✅ Prompts to test: Python, Java, Go, React
prompts = {
    "Python": "def fibonacci(n):",
    "Java": "public class Factorial {\n    public static int factorial(int n) {",
    "Go": "package main\nimport \"fmt\"\n\nfunc isPrime(n int) bool {",
    "React (JSX)": "import React, { useState } from 'react';\n\nfunction Counter() {"
}

# ✅ Run generation for each language
for lang, prompt in prompts.items():
    print(f"\n🧠 {lang} Prompt:\n{prompt}")
    code = generate_code(prompt)
    print(f"\n💻 {lang} Output:\n{code}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.5 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/12.1G [00:00<?, ?B/s]


🧠 Python Prompt:
def fibonacci(n):

💻 Python Output:
def fibonacci(n):
    if n<=1:
        return n
    return fibonacci(n-1)+fibonacci(n-1)

print(fibonacci(4))


# +
#factorial of a number
def factorial(n):
    if n<=1:
        return 1
    return n*factorial(n-1)

print(factorial(4))
# -

# # 2. Iterative functions

# +
#sum of n numbers
number=int(input("enter how many number you want to add"))
sum=0

for

🧠 Java Prompt:
public class Factorial {
    public static int factorial(int n) {

💻 Java Output:
public class Factorial {
    public static int factorial(int n) {
        if (n == 1) {
            return 1;
        }
        return n * factorial(n - 1);
    }

    public static void main(String[] args) {
        int result = factorial(5);
        System.out.println(result);
    }
}
/src/com/company/algorithms/leetcode/LongestCommonPrefix.java
package com.company.algorithms.leetcode;

/**
 * Created by  on 2019-02-10.
 * <p>
 * Write a function to find the longest

🧠 Go Prompt:
